In [1]:
import pandas as pd
import spacy

from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin

In [2]:
df = pd.read_csv('../../../../raw_data/data_prep.csv')

In [3]:
train, dev = train_test_split(df, test_size=0.1, random_state=42)

In [4]:
train.shape, dev.shape

((1080, 8), (120, 8))

In [5]:
categories = ['BB', 'BB+', 'BBB', 'BBB+', 'B-', 'B+', 'BB-', 'B', 'BBB-', 'C']

In [6]:
dev = dev.query('rating in @categories')

In [7]:
train = train.query('rating in @categories')

In [8]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [9]:
nlp = spacy.load('ru_core_news_lg')

In [10]:
def create_docbin(data):
    db = DocBin()
    global categories
    for i in range(data.shape[0]):    
        doc = nlp.make_doc(str(data["tokenized_str"][i]))
        doc.cats = {category: 0 for category in categories}
        doc.cats[data["rating"][i]] = 1
        db.add(doc)
    return db

In [11]:
create_docbin(train).to_disk("train.spacy")
create_docbin(dev).to_disk("dev.spacy")

In [13]:
! python -m spacy train config.cfg --output ./subsym_model --paths.train train.spacy --paths.dev dev.spacy

ℹ Saving to output directory: ..\subsym\sym_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       46.21    0.46
  0     200          17.93       52.86    0.53
  0     400          15.84       53.92    0.54
  1     600           8.52       63.26    0.63
  1     800          10.10       64.80    0.65
  2    1000           6.01       62.65    0.63
  2    1200           5.49       64.65    0.65
  3    1400           3.15       62.22    0.62
  3    1600           3.08       66.35    0.66
  4    1800           2.52       64.14    0.64
  4    2000           1.99       65.19    0.65
  5    2200           2.43       66.46    0.66
  5    2400           1.50       64.